# 감성 분류 모델 구축 (긍정, 부정)

In [1]:
import pandas as pd

In [2]:
train_df = pd.read_csv('ratings_train.txt', encoding='utf8', sep='\t', engine='python')

In [3]:
train_df.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [5]:
train_df.info() #데이터 정보 확인. document 컬럼에 mull이 5개 있음

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [6]:
#document 컬럼에서 null 제거
train_df = train_df[train_df['document'].notnull()]

In [7]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 149995 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        149995 non-null  int64 
 1   document  149995 non-null  object
 2   label     149995 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.6+ MB


In [9]:
#긍정 부정을 구분해주는 label 컬럼 확인 (긍정은 1, 부정은 0)
train_df['label'].value_counts()

0    75170
1    74825
Name: label, dtype: int64

In [15]:
#한긍 이외의 문자는 공백으로 변환(정규 표현식 이용)
import re

train_df['document'] = train_df['document'].apply(lambda x : re.sub(r'[^ ㄱ-ㅣ가-힣]+', " ",x))

In [16]:
train_df.head()

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다 평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화 스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [17]:
#평가용 데이터 파일인 ratings_test 텍스트 파일 준비
test_df = pd.read_csv('ratings_test.txt', encoding='utf8', sep='\t', engine='python')

In [18]:
test_df.head()

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


In [19]:
#테스트 데이터의 정보 확인
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        50000 non-null  int64 
 1   document  49997 non-null  object
 2   label     50000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.1+ MB


In [20]:
#document 컬럼의 null인 샘플 5개 제거
test_df = test_df[test_df['document'].notnull()]

In [21]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49997 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        49997 non-null  int64 
 1   document  49997 non-null  object
 2   label     49997 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.5+ MB


In [22]:
test_df['label'].value_counts()

1    25171
0    24826
Name: label, dtype: int64

In [23]:
#한글 이외의 문자는 공백으로 변환
test_df['document'] = test_df['document'].apply(lambda x : re.sub(r'[^ ㄱ-ㅣ가-힣]+', " ",x))

In [24]:
test_df.head()

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,,0
2,8544678,뭐야 이 평점들은 나쁘진 않지만 점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임 돈주고 보기에는,0
4,6723715,만 아니었어도 별 다섯 개 줬을텐데 왜 로 나와서 제 심기를 불편하게 하죠,0


## 데이터 준비 및 탐색 끝
## ==========================================

## 감성 분석 모델 구축

In [25]:
# 피터 벡터화 : TF-IDF
# 형태소를 분석하여 토큰화 --> 한글 형태소 엔진으로 Okt 이용
from konlpy.tag import Okt

okt=Okt()

In [26]:
def okt_tokenizer(text):
    tokens = okt.morphs(text)
    return tokens

In [29]:
# TF-IDF 기반 피처 벡터 생성
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(tokenizer = okt_tokenizer, ngram_range=(1,2), min_df=3, max_df=0.9 )
tfidf.fit(train_df['document'])
train_tfidf = tfidf.transform(train_df['document'])

C:\Users\gf218\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:484: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [30]:
# 로지스틱 회귀를 이용한 이진 분류
# 로지스틱 회귀 기반 분석 모델 생성
from sklearn.linear_model import LogisticRegression
SA_lr = LogisticRegression(random_state=0, max_iter=500)

In [32]:
SA_lr.fit(train_tfidf, train_df['label'])

LogisticRegression(max_iter=500, random_state=0)

In [34]:
# 로지스틱 회귀의 베스트 하이퍼파라미터 찾기
from sklearn.model_selection import GridSearchCV

params = {'C': [1,3,3.5,4,4.5,5] }
SA_lr_grid_cv = GridSearchCV(SA_lr, param_grid=params, cv=3, scoring='accuracy', verbose=1)

In [35]:
# 최적 분석 모델 훈련
SA_lr_grid_cv.fit(train_tfidf, train_df['label'])

Fitting 3 folds for each of 6 candidates, totalling 18 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  18 out of  18 | elapsed:  1.4min finished


GridSearchCV(cv=3, estimator=LogisticRegression(max_iter=500, random_state=0),
             param_grid={'C': [1, 3, 3.5, 4, 4.5, 5]}, scoring='accuracy',
             verbose=1)

In [36]:
print(SA_lr_grid_cv.best_params_, round(SA_lr_grid_cv.best_score_, 4))

{'C': 3} 0.8553


In [37]:
# 최적 파라미터의 베스트 모델 저장
SA_lr_best = SA_lr_grid_cv.best_estimator_

In [41]:
# 평가용 데이터의 피쳐 벡터화
test_tfidf = tfidf.transform(test_df['document'])

In [42]:
test_predict = SA_lr_best.predict(test_tfidf)

In [43]:
from sklearn.metrics import accuracy_score
print('감성 분석 정확도 : ', round(accuracy_score(test_df['label'], test_predict), 3))

감성 분석 정확도 :  0.858


# ===================================
# 네이버 웹툰 댓글 긍정, 부정 감성 분석 진행

## 3개로 구성된 댓글 파일 하나로 먼저 만들기

In [65]:
data1=pd.read_csv('comment_4to0.csv', encoding='utf-8-sig', engine='python')

In [68]:
data2=pd.read_csv('comment_16to4.csv', encoding='utf-8-sig', engine='python')

In [71]:
data1 = data1.append(data2, ignore_index=True)

In [75]:
data3=pd.read_csv('comment_last_to16.csv', encoding='utf-8-sig', engine='python')

In [79]:
data = data1.append(data3, ignore_index=True)

In [87]:
data.to_csv('comment_all_전독시.csv',encoding='utf-8-sig', index = False) 

## 데이터 준비 및 탐색

In [88]:
del data['Unnamed: 0']

In [89]:
data.rename(columns = {'0':'comment'}, inplace = True)

In [91]:
data['comment'].nunique()

118234

In [92]:
data.drop_duplicates(subset=['comment'], inplace=True)

In [94]:
data['comment'] = data['comment'].apply(lambda x : re.sub(r'[^ ㄱ-ㅣ가-힣]+', " ", x))

In [99]:
data.head()

,comment
0,그림 체고
1,배틀로얄 인가 ㅋ
2,도깨비 얼굴이 ㅎㄷㄷ
3,ㄱㅡ림 미쳤고 도깨비 아니라고 할 때 음성지원되누
4,이지혜 우리엄마 이름인데


In [111]:
import numpy as np
data['comment'].replace(' ', np.nan, inplace=True)

In [114]:
data = data.dropna(how='any')

In [118]:
len(data)

117918

## 감성 분석

In [119]:
# 분석할 댓글 데이터의 피쳐 벡터화   
data_tfidf = tfidf.transform(data['comment'])

In [120]:
# 최적 파라미터 학습 모델에 적용하여 감성분석
data_predict = SA_lr_best.predict(data_tfidf)

In [121]:
# 감성분석 결과값을 데이터 프레임에 저장
data['label'] = data_predict

In [122]:
data.head()

,comment,label
0,그림 체고,0
1,배틀로얄 인가 ㅋ,0
2,도깨비 얼굴이 ㅎㄷㄷ,1
3,ㄱㅡ림 미쳤고 도깨비 아니라고 할 때 음성지원되누,0
4,이지혜 우리엄마 이름인데,0


In [123]:
# csv 파일로 저장
data.to_csv('전독시감성분석결과.csv',encoding='utf-8-sig')

In [125]:
# 긍정 부정 컬럼 확인
data['label'].value_counts()

0    63147
1    54771
Name: label, dtype: int64

In [2]:
import pandas as pd
result_df = pd.read_csv("전독시감성분석결과.csv", encoding='utf-8-sig', engine='python')

In [3]:
del result_df['Unnamed: 0']

In [4]:
result_df.head()

,comment,label
0,그림 체고,0
1,배틀로얄 인가 ㅋ,0
2,도깨비 얼굴이 ㅎㄷㄷ,1
3,ㄱㅡ림 미쳤고 도깨비 아니라고 할 때 음성지원되누,0
4,이지혜 우리엄마 이름인데,0


In [5]:
# 감성 분석한 결과를 긍정과 부정으로 나누어서 저장
columns_name = ['comment','label']
negative_data = pd.DataFrame(columns=columns_name)
positive_data = pd.DataFrame(columns=columns_name)

for i, df in result_df.iterrows():
    comment = df["comment"]
    label = df["label"]
    
    # 부정 감성만 추출
    if label == 0:
        negative_data = negative_data.append(pd.DataFrame([[comment, label]], columns = columns_name), ignore_index=True)
    else: # 긍정 감성만 추출   
        positive_data = positive_data.append(pd.DataFrame([[comment, label]], columns = columns_name), ignore_index=True)


In [6]:
# 파일에 저장
negative_data.to_csv('전독시_부정감성_결과.csv',encoding='utf-8-sig')
positive_data.to_csv('전독시_긍정감성_결과.csv',encoding='utf-8-sig')